In [1]:
import numpy as np
import cv2
import ee
import IPython.display as disp

# ee.Authenticate()
ee.Initialize()

## Download NAIP high-altitude aerial image if necessary

In [4]:
coords = geojson_castaic['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

EEException: Image.clip: Parameter 'input' is required.

## Create a mosaic from the NAIP tiles to visualize quickly

In [ ]:
collection = ee.ImageCollection('USDA/NAIP/DOQQ').filterBounds(aoi).filterDate(ee.Date(startDate), ee.Date(endDate))

In [ ]:
mosaic = collection.mosaic().clip(aoi)

In [ ]:
url = mosaic.select(['R', 'G', 'B']).getThumbURL({'scale' : 10})
disp.Image(url=url, width=800)

## Download segmentation maps from DYNAMICWORLD

In [11]:


# startDate = '2023-03-01'
# endDate = '2023-03-26'

startDate = '2022-12-18'
endDate = '2022-12-23'
annotation_collection = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterBounds(aoi).filterDate(ee.Date(startDate), ee.Date(endDate))

In [12]:
annotation_mosaic = annotation_collection.mosaic().clip(aoi)
annotation_mosaic.bandNames().getInfo()

url = annotation_mosaic.select(['water']).getThumbURL({'scale' : 5, 'min':0, 'max':10})
disp.Image(url=url, width=800)

In [13]:
annotation_mosaic.bandNames().getInfo()

['water',
 'trees',
 'grass',
 'flooded_vegetation',
 'crops',
 'shrub_and_scrub',
 'built',
 'bare',
 'snow_and_ice',
 'label']

### Download all labels (class probabilites + mask)?

In [14]:
import time
num_images = annotation_collection.size().getInfo()
annotation_list = annotation_collection.toList(num_images)

for i in range(num_images):
    img = ee.Image(annotation_list.get(i)).clip(aoi).float()
    date_str = str(ee.Date(img.get("system:time_start")).format('YYYY-MM-dd').getInfo())

    idx_str = str(i).zfill(5)
    cur_fname = 'castaic_lake' + '_' + idx_str + '_' + date_str
    print(cur_fname)
    ee.batch.Export.image.toDrive(
        image=img,
        region=aoi,
        crs='EPSG:32611',
        description=cur_fname,
        folder='castaic_lake/prob',
        maxPixels=3e10,
    ).start()

    if i == 0:
        time.sleep(15) # pause to allow time for the first to complete - otherwise can get duplicate folders. 


castaic_lake_00000_2022-12-19


In [15]:
ee.data.listOperations()


[{'name': 'projects/earthengine-legacy/operations/L3SAA5PBHJCTISYHTS5QY3DB',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'SUCCEEDED',
   'description': 'castaic_lake_00000_2022-12-19',
   'createTime': '2023-05-09T04:51:01.220979Z',
   'updateTime': '2023-05-09T05:08:38.568071Z',
   'startTime': '2023-05-09T04:51:09.931270Z',
   'endTime': '2023-05-09T05:08:38.568071Z',
   'type': 'EXPORT_IMAGE',
   'destinationUris': ['https://drive.google.com/#folders/1xAoFqK_rlaWBoIDOWHgLgLNecYDWjkdw'],
   'attempt': 1,
   'progress': 1,
   'stages': [{'displayName': 'Create Local Files',
     'completeWorkUnits': 1,
     'totalWorkUnits': '1',
     'description': 'Computation and writing of temporary files.'},
    {'displayName': 'Write Files to Destination',
     'completeWorkUnits': 1,
     'totalWorkUnits': '1',
     'description': 'Uploading of files to the export destination.'}],
   'batchEecuUsageSeconds': 1543.976318359375},
  'done